# `PyTorch`'s `backward()` function and the vector-Jacobian product
The  goal of this notebook is to understand how the `PyTorch` `backward()` function performs (automatic) differentiation of functions.

# TLDR
The important point is as follows:
> `PyTorch`'s `backward()` function computes vector-Jacobian products.  By choosing the appropriate vector, which corresponds to the `gradient` argument in `backward()`, one can compute derivatives and gradients of functions.

The rest of this notebook is structured as follows:
- examples of usage without explanation;
- correspondence between mathematical differentiation and `PyTorch`'s `backward()` function; and  
- examples revisited, with justifications.

# References
- [The gradient argument in pytorch's backward function explained by examples](https://medium.com/@zhang_yang/the-gradient-argument-in-pytorchs-backward-function-explained-by-examples-68f266950c29) (Zhang Yang on Medium)
- [Computing gradients with backpropagation](https://www.cs.princeton.edu/courses/archive/fall18/cos324/files/backprop.pdf) (Ryan Adams, Princeton)

---
tags: pytorch, tutorial, backward, automatic differentiation, vector-Jacobian product, mathematics

# Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
%matplotlib inline

# Examples

## Compute the derivative of $f(x)\in \mathbb R$ evaluated at a single $x\in\mathbb R$
Evaluate the derivative of $f(x)=x^2$ at $x=1$:

In [2]:
x = torch.tensor(1., requires_grad=True)
y = x**2
y.backward()
x.grad

tensor(2.)

## Compute the derivative of $f(x)\in\mathbb R$ evaluated at $x_i\in\mathbb R$, $i=1, \dots, n$

In [3]:
x = torch.linspace(-1, 1, 5, requires_grad=True)
y = x**2
v = torch.ones_like(y)
y.backward(v)
x.grad

tensor([-2., -1.,  0.,  1.,  2.])

## Compute the gradient of $f(x)\in\mathbb R$ evaluated at a single $x\in\mathbb R^n$

In [4]:
x = torch.tensor([-1., 2.], requires_grad=True)
w = torch.tensor([3., 5.])
y = (x*x*w).sum()
y.backward()
x.grad

tensor([-6., 20.])

## Compute the gradient of $f(x)\in\mathbb R$  at $x_i\in\mathbb R^n$, $i=1\,, \dots\,, n$

In [5]:
x = torch.arange(6, dtype=float).view(3, 2)
x.requires_grad = True
w = torch.tensor([-1, 1])
y = (x*w).sum(1)
y.backward(torch.ones_like(y))
x.grad

tensor([[-1.,  1.],
        [-1.,  1.],
        [-1.,  1.]], dtype=torch.float64)

## Compute the derivative of $f(x)\in \mathbb R^m$ evaluated at a single $x\in\mathbb R$
Evaluate the derivative of $f(x)= (-x^3, 5x)$ evaluated at $x=1$: 

In [6]:
x = torch.tensor(1., requires_grad=True)
y = torch.stack([-x**3, 5*x]).view(1, -1)
print(f"y({x.data.item()})    = {list(y.data.numpy()[0,:])}")

v1 = torch.tensor([[1., 0.]])
y.backward(v1, retain_graph=True)
print(f"y_1'({x.data.item()}) = {x.grad.data.item():>4}")

x.grad.zero_()

v2 = torch.tensor([[0., 1.]])
y.backward(v2, retain_graph=True)
print(f"y_2'({x.data.item()}) = {x.grad.data.item():>4}")

y(1.0)    = [-1.0, 5.0]
y_1'(1.0) = -3.0
y_2'(1.0) =  5.0


# The correspondence between mathematical differentiation and `PyTorch`'s `backward()` function
It is useful to draw a careful analogy between terminology which is conventional in mathematics and that used in `PyTorch`.

## Warning: "dimension"
The notion of dimension in mathematics is not the same as that in `PyTorch`.

## Preliminaries
### Mathematical notation

- The set of real values is denoted by $\mathbb R$.  
- The set of **vectors of dimension $n$**, such as $x=(x_1, \dots, x_n)$, is denoted $\mathbb R^n$.  Vectors correspond to `0`-dimensional `tensor`s.
- The set of **$m\times n$ matrices**, such as
$\begin{bmatrix}
    x_{1,1}&\dots&x_{1,n}\\
    \vdots&\ddots&\vdots\\
    x_{m,1}&\dots&x_{m,n}
\end{bmatrix}$,
is denoted
$\mathbb R^{m\times n}$.
- The set of **row vectors**, i.e. are single-row matrices, is thus $\mathbb R^{1,n}$.
- The set of **column vectors**, i.e. single-column matrices, is thus $\mathbb R^{m,1}$.  

The three sets $\mathbb R^n$, $\mathbb R^{1,n}$, and $\mathbb R^{n,1}$, are distinct although it is common to identify them.

Observe that vectors and row vectors can be distinguished by their notation: the vector $(x_1,\dots, x_n)\in\mathbb R^n$ is not the row vector $\left[x_1,\dots, x_n\right]\in\mathbb R^{1,n}
$.


### `PyTorch` `tensor`s
- By `tensor`, we mean a `PyTorch` object of the class `torch.tensor`.  Its shape `torch.Size([m,n])` will simply be denoted `[m,n]`.
- Real scalars correspond to `0`-dimensional `tensor`s.
- Vectors correspond to `1`-dimensional `tensor`s.
- Row vectors correspond to `2`-dimensional `tensor`s with shape `[1,2]`.
- Column vectors correspond to `2`-dimensional `tensor`s with shape `[2,1]`.


### Prerequisites on differentiation

We do not recall the definitions for:
- the **derivative** $f'(x)$ of a scalar, uni-variate function $f(x)$, $x\in\mathbb R$;
- the **partial derivatives** $\frac{\partial f}{\partial x_i}$, $i=1, \dots, n$, of a scalar, multi-variate function $f(x)\in \mathbb R$, $x\in\mathbb R^n$;
- **Einstein's convention** of  summing over repeated indices.

In particular, when we speak of the derivative, it is implicit that the function is univariate, i.e. it depends on a single (real) variable $x$.

### Derivatives of vector-valued functions $f(x)\in\mathbb R^m$, $x\in\mathbb R$
The **derivative** of a vector-valued function $f(x)=(f_1(x), \dots, f_m(x))\in\mathbb R^m$ is the column vector of the derivatives of the component functions:
$$\begin{bmatrix}f_1'(x)\\\vdots\\f_m'(x)\end{bmatrix}\in\mathbb R^{m\times 1}\,.$$

### Gradients of scalar-valued functions $f(x)\in\mathbb R$, $x\in\mathbb R^n$
The **gradient** of a scalar-valued function $f(x)\in\mathbb R$, $x\in\mathbb R^n$, is the *row* vector of its partial derivatives:
$$\nabla f(x) = \left[\frac{\partial f}{\partial x_1}(x)\,,\dots\,,\frac{\partial f}{\partial x_n}(x)\right]\in\mathbb R^{1\times n}\,.$$

### Jacobians
The **Jacobian** of a vector-valued function $f(x)=(f_1(x)\,, \dots\,, f_m(x))\in\mathbb R^m$, $x\in\mathbb R^n$, is the vertical concatenation of the gradients of the component functions $f_1, \dots, f_m$:
$$J_f(x)=\begin{bmatrix}
    \frac{\partial f_1}{\partial x_1}(x)&\dots&\frac{\partial f_1}{\partial x_n}(x)\\
    \vdots&\ddots&\vdots\\
    \frac{\partial f_m}{\partial x_1}(x)&\dots&\frac{\partial f_m}{\partial x_n}(x)
\end{bmatrix}\in\mathbb R^{m\times n}\,.
$$
#### The case $m=1$
In case $m=1$, the Jacobian agrees with the gradient.

### Vector-Jacobian products
Given a row vector $v\in\mathbb R^{1\times m}$, the **vector-Jacobian product** is the row vector
$$v J_f(x) = \left[
v_j\frac{\partial f_j}{\partial x_1}(x)\,, \dots\,, v_j\frac{\partial f_j}{\partial x_n}(x)
\right]\in\mathbb R^{1\times n
}\,,
$$
where we have used the convention of summing over repeated indices.


In particular, if $v$ is the gradient of a scalar-valued function $z=l(y)$ evaluated at $f(x)$, i.e. $v = \nabla l(y)$ where $y=f(x)$, then
$$v J_f(x) = \left[\frac{\partial l}{\partial y_j}(y)\frac{\partial f_j}{\partial x_1}(x)\,, \dots\,,
\frac{\partial l}{\partial y_j}(y)\frac{\partial f_j}{\partial x_m}(x)\right]
\,=\,\nabla (l\circ f)(x)\in\mathbb R^{1\times n}\,.$$

###  `PyTorch`'s `backward()` function
With `x` denoting the input `tensor` and `y` denoting the output `tensor`, we will call the `backward()` method on `y` to compute the (partial) derivatives at `x`.  These derivatives will be collected ("accumulated") in the attribute `x.grad`.

`PyTorch`'s `backward()` function, on the other hand, computes vector-Jacobian products.  To this end, we need to determine what corresponds to the row vector $v$ above.

### The `gradient` argument in `backward()`
The `gradient` argument in the function `backward()` corresponds to the row vector $v$ in the vector-Jacobian product introduced above.

`PyTorch`'s `backward()` function takes an argument `gradient` which corresponds to the row vector $v$ above.  It takes a default value of `torch.tensor(1.)`.

If `y` is `0`-dimensional or `1`-dimensional with shape `[1,1]`, `gradient` can be `0`-dimensional or `1`-dimensional with shape `[1,1]`.  In particular, if `y` corresponds to the output of a scalar function $f(x)$, regardless of whether uni-variate or multi-variate, the `gradient` argument can be ignored.

Otherwise `v` must have same shape as `y`.

# Examples revisited

## Compute the derivative of $f(x)\in\mathbb R$ at a single $x\in\mathbb R$
The notation implies that the function is uni-variate (i.e. it depends on a single variable) and is scalar (
i.e. it returns a single value).

In [7]:
x = torch.tensor(1., requires_grad=True)
y = x**2
y.backward()
x, y, x.grad

(tensor(1., requires_grad=True),
 tensor(1., grad_fn=<PowBackward0>),
 tensor(2.))

Specifying the `gradient` and assigning it a value of `1.` will return the same result:

In [8]:
x = torch.tensor(1., requires_grad=True)
y = x**2
v = torch.tensor(1.)
y.backward(v)
x, y, x.grad

(tensor(1., requires_grad=True),
 tensor(1., grad_fn=<PowBackward0>),
 tensor(2.))

Here, `x` or `v` can be replaced with `torch.tensor([1.])`.

## Compute the derivative of $f(x)\in\mathbb R$ at $x_i\in\mathbb R$, $i=1, \dots, n$
To evaluate the derivative of a scalar, univariate function $f(x)\in\mathbb R$ at multiple values $x_1, \dots, x_n\in\mathbb R$, we create a new, multi-variate *and* vector-valued function of $x = (x_1,\dots, x_n)\in\mathbb R^n$:
$$F(x)=\left[f(x_1)\,,\dots\,, f(x_n)\right]\,.$$
Thus, its Jacobian is
$$J_F(x)=\begin{bmatrix}
f'(x_1)&&&&\\
&\ddots&&&\\
&&f'(x_j)&&\\
&&&\ddots&\\
&&&&f'(x_n)\end{bmatrix}
$$
where the off-diagonal terms are all $0$.
Thus, with the row vector of $1$'s, $v=\mathbb 1_{1\times n}\in\mathbb R^{1\times n}$, we obtain the gradient of $f$ evaluated at $x_1\,, \dots\,, x_n$:
$$\left[f'(x_1)\,,\dots\,, f'(x_j)\,, \dots\,, f'(x_n)\right] 
=\left[1\,,\dots\,,1\right]
\begin{bmatrix}
f'(x_1)&&&&\\
&\ddots&&&\\
&&f'(x_j)&&\\
&&&\ddots&\\
&&&&f'(x_n)\end{bmatrix}
= \mathbb 1_{\mathbb R^{1\times n}}J_F(x)\,.$$

In [9]:
x = torch.linspace(-1, 1, 5, requires_grad=True)
y = x**2
v = torch.ones_like(y)
y.backward(v)
x, y, x.grad

(tensor([-1.0000, -0.5000,  0.0000,  0.5000,  1.0000], requires_grad=True),
 tensor([1.0000, 0.2500, 0.0000, 0.2500, 1.0000], grad_fn=<PowBackward0>),
 tensor([-2., -1.,  0.,  1.,  2.]))

Remarks:
- `y` corresponds to $F(x)$;
- `v` must have same shape as `y`;
- `x` and `x.grad` have same shape.

## Compute the gradient of $f(x)\in\mathbb R$ at a single $x\in\mathbb R^n$
Since the function is scalar, the `gradient` argument can be ignored:

In [10]:
x = torch.tensor([-1., 2.], requires_grad=True)
w = torch.tensor([3., 5.])
y = (x*x*w).sum()
y.backward()
x, y, x.grad

(tensor([-1.,  2.], requires_grad=True),
 tensor(23., grad_fn=<SumBackward0>),
 tensor([-6., 20.]))

## Compute the gradient of $f(x)\in\mathbb R$ at $x_i\in\mathbb R^n$, $i=1, \dots, n$

Here, `y` corresponds to $F(x)=(f(x_1), \dots, f(x_n))\in\mathbb R^m$, where $m=3$ and $n=2$, and noting that `x` has shape `[m,n]`.  In turn, the `gradient` argument corresponds to $\left[1,\dots,1\right]=\mathbb 1_{\mathbb R^{1,n}}$.

In [15]:
x = torch.arange(6, dtype=float).view(3, 2)
x.requires_grad = True
w = torch.tensor([-1, 1])
y = (x*w).sum(1)
y.backward(torch.ones_like(y))
x, y, x.grad

(tensor([[0., 1.],
         [2., 3.],
         [4., 5.]], dtype=torch.float64, requires_grad=True),
 tensor([1., 1., 1.], dtype=torch.float64, grad_fn=<SumBackward1>),
 tensor([[-1.,  1.],
         [-1.,  1.],
         [-1.,  1.]], dtype=torch.float64))

## Compute the derivative of $f(x)\in\mathbb R^m$ at a single $x\in\mathbb R$
There are two complications.
1. The derivative for each component function $f_1, \dots, f_m$ of $f(x)$ is calculated one at a time, because `backward()` implements **reverse** differentiation.  In order to this, the flag `retain_graph` must be set to `True`. 
1. Since derivatives are accumulated in `x`, its `grad` attribute must be set to zero.

In [12]:
x = torch.tensor(1., requires_grad=True)
y = torch.stack([-x**3, 5*x]).view(1, -1)
print(f"y({x.data.item()})  = {list(y.data.numpy()[0,:])}")

v1 = torch.tensor([[1., 0.]])
y.backward(v1, retain_graph=True)
print(f"y_1'(1) = {x.grad.data.item():>4}")

x.grad.zero_()

v2 = torch.tensor([[0., 1.]])
y.backward(v2, retain_graph=True)
print(f"y_2'(1) = {x.grad.data.item():>4}")

y(1.0)  = [-1.0, 5.0]
y_1'(1) = -3.0
y_2'(1) =  5.0


# Further cases: input `tensor`s of higher dimensions

The above also works if $x$ is a matrix, or more generally a tensor.

In [13]:
x = torch.tensor([[1., 2.], [3., 4.]], requires_grad=True)
w = torch.ones_like(x)
y = (x*w).sum()
y.backward()
x, y, x.grad

(tensor([[1., 2.],
         [3., 4.]], requires_grad=True),
 tensor(10., grad_fn=<SumBackward0>),
 tensor([[1., 1.],
         [1., 1.]]))

In [14]:
x_min, x_max, y_min, y_max = -4, 4., -4., 4.

linear = nn.Linear(2, 1)
linear.bias.data = torch.tensor([0.])

N = 9
Y, X = np.mgrid[y_min:y_max:N*1j, x_min:x_max:N*1j]
X = torch.tensor(X, dtype=torch.float32, requires_grad=True)
Y = torch.tensor(Y, dtype=torch.float32, requires_grad=True)
XY = torch.stack([X, Y], -1).reshape(N*N,2)

Z = linear(XY).sigmoid()
Z.backward(torch.ones_like(Z))
gradf_X, gradf_Y = X.grad, Y.grad

X.shape, gradf_X.shape, Y.shape, gradf_Y.shape, XY.shape, Z.shape

(torch.Size([9, 9]),
 torch.Size([9, 9]),
 torch.Size([9, 9]),
 torch.Size([9, 9]),
 torch.Size([81, 2]),
 torch.Size([81, 1]))